## Download github repo

In [20]:

#In case you want to reupload ditto if you have made changes in the github
#import shutil
#shutil.rmtree("/kaggle/working/ditto")
#%cd /kaggle/working/
#!git clone https://github.com/nhvd3500111/ditto
#%cd /kaggle/working/ditto

#just clone the repo
!git clone https://github.com/nhvd3500111/ditto

## Install packages and fp16 optimization

In [2]:
# install requirements
!pip install -r requirements.txt
import nltk
nltk.download('stopwords')
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli
%cd /kaggle/working/ditto
!pip install transformers
!pip install tensorboardX
!pip install jsonlines
!pip install openpyxl

## Train DITTO

In [22]:
# Step 1:
# train the wdc model with the save_model flag on. This is the GRU Ditto Model
# it will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.

!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Structured/Fodors-Zagats \
  --batch_size 32 \
  --max_len 128 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm roberta \
  --fp16 \
  --save_model \
  --da drop_col \
  --neural gru


## Running the matcher

In [23]:
# Step 2: run the matcher

#da , dk and summarize must be the same as above trained
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task Structured/Fodors-Zagats \
  --input_path data/er_magellan/Structured/Fodors-Zagats/test.txt \
  --output_path output/output_small_1.jsonl \
  --lm roberta \
  --max_len 128 \
  --use_gpu \
  --fp16 \
  --da drop_col\
  --checkpoint_path checkpoints/ \
  --neural gru

#We will download the xlsx file manually from the output folder